In [49]:
!pip3 install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [50]:
!pip3 install seaborn


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [51]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [52]:
!pip3 install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [53]:
# Import torch and neural network library 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset  


# import sklearn model_selection, StandardScaler
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import numpy, pandas, matplotlib, seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Setup device either gpu or cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


print('Device used: ', device)

Device used:  cpu


In [54]:
# read the dataset framingham.csv and display the first 5 rows.
data = pd.read_csv('/Users/suarezcruzrosy/Downloads/framingham.csv')
data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [55]:
# display the shape, null values
data.shape, data.isnull().sum()


((4240, 16),
 male                 0
 age                  0
 education          105
 currentSmoker        0
 cigsPerDay          29
 BPMeds              53
 prevalentStroke      0
 prevalentHyp         0
 diabetes             0
 totChol             50
 sysBP                0
 diaBP                0
 BMI                 19
 heartRate            1
 glucose            388
 TenYearCHD           0
 dtype: int64)

In [56]:
# Fill null values with either median or mean.
data['education'] = data['education'].fillna(data['education'].median())
data['cigsPerDay'] = data['cigsPerDay'].fillna(data['cigsPerDay'].mean())
data['BPMeds'] = data['BPMeds'].fillna(data['BPMeds'].mean()) 
data['totChol'] = data['totChol'].fillna(data['totChol'].mean())
data['BMI'] = data['BMI'].fillna(data['BMI'].mean()) 
data['glucose'] = data['glucose'].fillna(data['glucose'].mean())
data['heartRate'] = data['heartRate'].fillna(data['heartRate'].mean())  
data.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [57]:
# get the features as X and the target as y
X_features = data.drop(['TenYearCHD'], axis=1)
y_target = data['TenYearCHD']

In [58]:
# Split the data into train and test:
x_train , x_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=42)

In [59]:
# Scale the features X_train and X_test using StandardScaler 
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)


# Convert the X_train, X_test, y_train, y_test to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)



In [60]:
# Build your neural network
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [61]:
# Instantiate the model, define the loss function and the optimizer
model = NeuralNetwork()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)    


In [62]:
# Train the model

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")  


In [63]:
!pip3 install --upgrade torch torchvision


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [64]:
# Evaluate the model 

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 28 * 28, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

model = SimpleModel().to(device)

loss_fn = nn.CrossEntropyLoss()

def test(dataloader, model, loss_fn):
    print("Testing...")
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    print(f"Dataset size: {size}, Number of batches: {num_batches}")
    
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for i, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            batch_loss = loss_fn(pred, y).item()
            test_loss += batch_loss
            batch_correct = (pred.argmax(1) == y).type(torch.float).sum().item()
            correct += batch_correct
            
            if i % 10 == 0:
                print(f"Batch {i} - Loss: {batch_loss:.4f}, Correct: {batch_correct:.4f}")
    
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

print("Calling test function...")
test(test_dataloader, model, loss_fn)




Using device: cpu
Calling test function...
Testing...
Dataset size: 10000, Number of batches: 157


RuntimeError: Numpy is not available